<a href="https://colab.research.google.com/github/herosunly/100-Days-Of-ML-Code/blob/master/CNN_metafeature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from sklearn.preprocessing import LabelBinarizer,LabelEncoder
from sklearn.model_selection import StratifiedKFold

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path = './drive/My Drive/data/'
import pickle
train_labels = pickle.load(open(path + 'train_label.pkl', 'rb'))
train_seq = pickle.load(open(path + 'train_seq.pkl', 'rb'))
test_seq = pickle.load(open(path + 'test_seq.pkl', 'rb'))

In [0]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Lambda, Embedding, Dropout, Activation,GRU,Bidirectional
from tensorflow.keras.layers import Conv1D,Conv2D,MaxPooling2D,GlobalAveragePooling1D,GlobalMaxPooling1D, MaxPooling1D, Flatten
from tensorflow.keras.layers import CuDNNGRU, CuDNNLSTM, SpatialDropout1D
from tensorflow.keras.layers import concatenate, Concatenate, Average, Dot, Maximum, Multiply, Subtract, average
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import SGD
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
from tensorflow.keras.layers import SpatialDropout1D

In [0]:
def TextCNN(max_len,max_cnt,embed_size,
            num_filters,kernel_size,
            conv_action,
            mask_zero):
    _input = Input(shape=(max_len,), dtype='int32')
    _embed = Embedding(max_cnt, embed_size, input_length=max_len, mask_zero=mask_zero)(_input)
    _embed = SpatialDropout1D(0.15)(_embed)
    warppers = []
    for _kernel_size in kernel_size:
        conv1d = Conv1D(filters=num_filters, kernel_size=_kernel_size, activation=conv_action)(_embed)
        warppers.append(GlobalMaxPooling1D()(conv1d))
                        
    fc = concatenate(warppers)
    fc = Dropout(0.5)(fc)
    #fc = BatchNormalization()(fc)
    fc = Dense(256, activation='relu')(fc)
    fc = Dropout(0.25)(fc)
    #fc = BatchNormalization()(fc) 
    preds = Dense(8, activation = 'softmax')(fc)
    
    model = Model(inputs=_input, outputs=preds)
    
    #model.compile(loss='categorical_crossentropy',
    #              optimizer='adam',
    #              metrics=['accuracy'])
    return model

In [0]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [0]:
max_len = 6000
max_cnt = 302
embed_size = 256
num_filters = 64
kernel_size = [2,4,6,8,10,12,14]
conv_action = 'relu'
mask_zero = False
TRAIN = True

In [0]:
import os
import tensorflow as tf
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
tf.logging.set_verbosity(0)
meta_train = np.zeros(shape = (len(train_seq),8))
meta_test = np.zeros(shape = (len(test_seq),8))
FLAG = True
print(train_seq.shape, test_seq.shape)
for i,(tr_ind,te_ind) in enumerate(skf.split(train_seq, train_labels.argmax(1))):
    print('FOLD: ', i)
    print(len(te_ind),len(tr_ind))
    model = TextCNN(max_len,max_cnt,embed_size,num_filters,kernel_size,conv_action,mask_zero)
    model_name = 'benchmark_textcnn_fold_'+str(i)
    X_train,X_train_label = train_seq[tr_ind],train_labels[tr_ind]
    X_val,X_val_label = train_seq[te_ind],train_labels[te_ind]
    model_save_path = path + 'model_weight_final/%s_%s.hdf5'%(model_name,embed_size)
    early_stopping = EarlyStopping(monitor='val_loss', patience=3)
    model_checkpoint = ModelCheckpoint(model_save_path, save_best_only=True, save_weights_only=True)

    strategy = tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR']))
    model = tf.contrib.tpu.keras_to_tpu_model(model, strategy)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    if TRAIN and FLAG:
        model.fit(X_train,X_train_label,
                  validation_data=(X_val,X_val_label),
                  epochs=100,batch_size=128,
                  shuffle=True,
                  callbacks=[early_stopping]
                 )
    
    #model.load_weights(model_save_path)
    model.save_weights('/tmp/%s.h5' % model_name, overwrite=True)
    cpu_model = model.sync_to_cpu()
    pred_val = cpu_model.predict(X_val, batch_size=64, verbose=1)
    pred_test = cpu_model.predict(test_seq, batch_size=64, verbose=1)
    
    meta_train[te_ind] = pred_val
    meta_test += pred_test
    tf.keras.backend.clear_session()
meta_test /= 5.0


(13887, 6000) (12955, 6000)
FOLD:  0
2780 11107
Train on 11107 samples, validate on 2780 samples
Epoch 1/100
11107/11107 [==============================] - 330s 30ms/step - loss: 0.9508 - acc: 0.6787 - val_loss: 0.5115 - val_acc: 0.8369
Epoch 2/100
11107/11107 [==============================] - 9s 855us/step - loss: 0.5170 - acc: 0.8372 - val_loss: 0.4298 - val_acc: 0.8578
Epoch 3/100
11107/11107 [==============================] - 9s 837us/step - loss: 0.4347 - acc: 0.8607 - val_loss: 0.4084 - val_acc: 0.8642
Epoch 4/100
11107/11107 [==============================] - 9s 850us/step - loss: 0.3895 - acc: 0.8759 - val_loss: 0.4254 - val_acc: 0.8621
Epoch 5/100
11107/11107 [==============================] - 9s 823us/step - loss: 0.3759 - acc: 0.8801 - val_loss: 0.4083 - val_acc: 0.8671
Epoch 6/100
11107/11107 [==============================] - 9s 845us/step - loss: 0.3435 - acc: 0.8857 - val_loss: 0.3672 - val_acc: 0.8851
Epoch 7/100
11107/11107 [==============================] - 9s 848us/

In [0]:
pd.to_pickle(meta_train, path + 'train_meta_cnn.pkl')
pd.to_pickle(meta_test, path + 'test_meta_cnn.pkl')

In [0]:
%pwd